In [ ]:
!pip install torch
!pip install transformers
!pip install kobert-transformers

In [ ]:
import pandas as pd
import re
import nltk
from kobert_transformers import get_tokenizer
from nltk.corpus import stopwords
from google.colab import drive

# NLTK stopwords 다운로드
nltk.download('stopwords')

# KoBERT 전용 토크나이저 초기화
tokenizer = get_tokenizer()

# Google Drive 마운트
drive.mount('/content/drive')

# 1. CSV 파일 불러오기
df = pd.read_csv('/content/drive/MyDrive/Team2/daum_news_data.csv')

# 2. 결측값 처리 (빈 문자열로 대체)
df['content'] = df['content'].fillna('')
df['author'] = df['author'].fillna('')
df['title'] = df['title'].fillna('')

# 3. content, title, author 클렌징

# 4. 숫자 제거 (content, title, author)
df['content'] = df['content'].str.replace(r'\d+', '', regex=True)
df['title'] = df['title'].str.replace(r'\d+', '', regex=True)
df['author'] = df['author'].str.replace(r'\d+', '', regex=True)

# 5. 특수문자 및 한문 제거 (content, title, author)
df['content'] = df['content'].apply(lambda x: re.sub(r'[^\w\s]|[\u4e00-\u9fff]', '', x))
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]|[\u4e00-\u9fff]', '', x))
df['author'] = df['author'].apply(lambda x: re.sub(r'[^\w\s]|[\u4e00-\u9fff]', '', x))

# 6. 중복 데이터 제거 (content, title 기준)
df = df.drop_duplicates(subset=['content', 'title'])

# 7. 불용어 제거 (NLTK와 KoBERT를 사용)
stop_words = set(stopwords.words('english'))  # NLTK의 영어 불용어 사용

def remove_stopwords(text):
    tokens = tokenizer.tokenize(text)  # KoBERT 토크나이저로 형태소 분석
    return ' '.join([token for token in tokens if token not in stop_words])  # NLTK 불용어 제거

df['content'] = df['content'].apply(remove_stopwords)
df['title'] = df['title'].apply(remove_stopwords)

# 8. KoBERT에서 남는 특수문자 및 공백 처리
df['content'] = df['content'].apply(lambda x: re.sub(r'[^\w\s]|▁', '', x))  # 특수문자 및 '▁' 제거
df['title'] = df['title'].apply(lambda x: re.sub(r'[^\w\s]|▁', '', x))  # 특수문자 및 '▁' 제거

# 9. '기자' 제거 (author 열에서)
df['author'] = df['author'].str.replace('기자', '', regex=False).str.strip()

# 10. 영어 제거 (content, title, author)
df['content'] = df['content'].str.replace(r'[a-zA-Z]', '', regex=True)
df['title'] = df['title'].str.replace(r'[a-zA-Z]', '', regex=True)
df['author'] = df['author'].str.replace(r'[a-zA-Z]', '', regex=True)

# 11. 중복된 데이터 제거 (title 기준)
df = df.drop_duplicates(subset=['title'])

# 12. 불필요한 공백 제거 (content, title, author)
df['content'] = df['content'].str.strip()  # 앞뒤 공백 제거
df['content'] = df['content'].str.replace(r'\s+', ' ', regex=True)  # 다중 공백 단일화
df['title'] = df['title'].str.strip()  # 앞뒤 공백 제거
df['title'] = df['title'].str.replace(r'\s+', ' ', regex=True)  # 다중 공백 단일화
df['author'] = df['author'].str.strip()  # 앞뒤 공백 제거
df['author'] = df['author'].str.replace(r'\s+', ' ', regex=True)  # 다중 공백 단일화

# 13. title에 공백 또는 빈 값만 있을 경우 해당 행 제거
df = df[df['title'].str.strip() != '']

# 클렌징 완료 후 확인
print(df.head(10))

# 클렌징된 데이터 저장 (Google Drive의 'MyDrive' 폴더에 저장)
df.to_csv('/content/drive/MyDrive/Team2/daum_news_data_cleaned.csv', index=False)
